# Forensische Datenanalyse von Weblogs mit DuckLake (DuckDB)

Dieses Notebook dokumentiert die forensische Untersuchung von Weblog-Daten zur Identifizierung und zum Beweis einer simulierten **Datenmanipulation**.

**DuckLake** (eine Erweiterung für DuckDB) wird verwendet. DuckLake erstellt für jede Schreiboperation einen unveränderlichen **Snapshot** (Version). Hierfür wird das **Time Travel**-Feature von DuckLake genutzt, um unveränderliche Snapshots früherer Datenzustände zu rekonstruieren und so den Originalzustand mit dem manipulierten Zustand zu vergleichen (https://ducklake.select/docs/stable/duckdb/usage/time_travel.html)

## Zweck und Ziel der Untersuchung

Die Untersuchung verfolgt das primäre Ziel, die forensische Nachweisbarkeit von Datenmanipulation in Weblogs zu demonstrieren, indem das Time Travel-Feature von DuckLake genutzt wird, um manipulierte Datensätze anhand unveränderlicher Snapshots zu rekonstruieren und zu vergleichen.

Ein weiteres Ziel ist der Performance-Vergleich von vier relevanten kryptografischen Verfahren (SHA-256, SHA3-256, AES-256, Blowfish), um ihre Eignung für die Datenintegrität (Hashing) und den vertraulichen Datenaustausch (Verschlüsselung) in realen Szenarien zu bewerten.

Dabei wird die Integrität der Daten durch Hashing-Verfahren verifiziert, indem bereits kleinste Änderungen am Datensatz zu fundamental unterschiedlichen Hash-Werten führen, was als sofortiger Manipulationsnachweis dient.

Parallel dazu wird die Vertraulichkeit der Daten durch die symmetrische Verschlüsselung gewährleistet und die Performance von modernen (AES) und traditionellen (Blowfish) Algorithmen anhand der gemessenen Ver- und Entschlüsselungszeiten analysiert.

Zusammenfassend dient die Arbeit dazu, sowohl die technische Nachweisbarkeit von Cyber-Vorfällen als auch die fundierte Auswahl adäquater kryptografischer Schutzmaßnahmen in einer datenbankbasierten Umgebung zu evaluieren.

## 1. Vorbereitung und Daten-Setup

### Importe einfügen

In [1]:
!pip install duckdb
!pip install pandas



In [2]:
import duckdb
import os
import time
from datetime import datetime

Der Code richtet eine persistente DuckDB-Datenbank ein und erstellt separate Verzeichnisse für Metadaten und Daten, um eine saubere Trennung zu gewährleisten. Anschließend wird die DuckLake-Erweiterung installiert, geladen und ein Katalog mit dem definierten Speicherort angebunden. Zum Schluss wird dieser Katalog als Standard gesetzt, sodass alle weiteren SQL-Operationen automatisch darin ausgeführt werden.

In [3]:
# 1. Konfiguriere die Pfade
# Verzeichnis für Metadaten und ein separates für die Daten (best practice)
DUCKLAKE_METADATA_PATH = 'ducklake_metadata.ducklake'
DUCKLAKE_DATA_PATH = 'ducklake_data_files'
CATALOG_NAME = 'weblog_lake'

# Verzeichnisse erstellen, falls sie noch nicht existieren
os.makedirs(DUCKLAKE_DATA_PATH, exist_ok=True)

# 2. Verbinde mit DuckDB
# Die Verbindung zu einer permanenten DB ist für Langlebigkeit der Demo besser
con = duckdb.connect(database='weblog_demo.duckdb')

# DuckLake-Erweiterung
con.sql("INSTALL ducklake;")
con.sql("LOAD ducklake;")

# 4. Attach DuckLake-Katalog
attach_query = f"""
    ATTACH 'ducklake:{DUCKLAKE_METADATA_PATH}' AS {CATALOG_NAME} (DATA_PATH '{DUCKLAKE_DATA_PATH}');
"""
con.sql(attach_query)

# 5. DuckLake-Katalog als Standard setzen
con.sql(f"USE {CATALOG_NAME};")

print(f"✅ DuckLake '{CATALOG_NAME}' erfolgreich erstellt und verbunden.")


✅ DuckLake 'weblog_lake' erfolgreich erstellt und verbunden.


## Code


### Link zum Datensatz
https://www.kaggle.com/datasets/shawon10/web-log-dataset

Der Datensatz enthält Server-Logdaten des RUET Online Judge (RUET OJ), einem Online-Bewertungssystem einer Universität. Er umfasst 16.008 Einträge mit vier Spalten: IP-Adresse, Zeitstempel, aufgerufene URL und HTTP-Statuscode. Ziel ist es, das Nutzerverhalten und die Serveraktivität zu analysieren, z. B. Login-Vorgänge oder Seitenaufrufe.

In [4]:
! pip install -q kaggle

In [5]:
! mkdir ~/.kaggle

Übergabe von Kaggle Benutzer Daten:
{"username":"....","key":"...."}

In [6]:
# ... Hier Ihre JSON Cred als dictionary eingeben
d_json_cred ={"username":"lizzldizzl","key":"7126d6d48a18986c8a8704fbb94e4a44"}

Kaggle Zugangsdaten speichern



In [7]:
import pandas as pd
kaggle_cred = pd.DataFrame(d_json_cred, index=[0]).to_json("~/.kaggle/kaggle.json")

Authorisierung geben dass Kaggle Daten heruntergeladen werden dürfen

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shawon10/web-log-dataset

Unzip der Daten

In [ ]:
!unzip web-log-dataset.zip -d ./data

## 2. DuckLake-Initialisierung und Snapshot 1 (Basis-Beweis)



Die Rohdaten werden in Pandas geladen, bereinigt und anschließend in den DuckLake-Katalog als erster unveränderlicher Zustand (Snapshot 1) eingefügt. Dies sichert den Original-Beweis.

In [ ]:
import duckdb, os
import pandas as pd
import time
from datetime import datetime # Import datetime

# --- Pfade & Namen ---
LOG_PATH = './data/weblog.csv' # Pfad zur Logdatei
DUCKLAKE_METADATA_PATH = 'forensic_evidence.ducklake'
DUCKLAKE_DATA_PATH = 'forensic_data_files'
CATALOG_NAME = 'forensic_log_archive'
TABLE_NAME = 'access_logs'

# Ordner erstellen
os.makedirs(DUCKLAKE_DATA_PATH, exist_ok=True)

# --- Datenbereinigung und Transformation ---
try:
    # 1. Daten einlesen
    df = pd.read_csv(LOG_PATH)

    # 2. Spaltennamen standardisieren
    df.columns = df.columns.str.strip().str.lower()

    # 3. Zeitstempel konvertieren (entfernt '[', nutzt Apache Log Format)
    df['time'] = pd.to_datetime(
        df['time'].astype(str).str.replace("[", "", regex=False),
        format="%d/%b/%Y:%H:%M:%S",
        errors="coerce"
    )

    # 4. Spaltenumbenennung ('staus' -> 'status_code' und 'time' -> 'timestamp')
    df.rename(columns={'time': 'timestamp', 'staus': 'status_code'}, inplace=True)

    # 5. Statuscodes in robusten Integer-Typ konvertieren
    df['status_code'] = pd.to_numeric(
        df['status_code'],
        errors='coerce'
    ).astype(pd.Int64Dtype())

    # 6. Finale Spaltenauswahl für die Archivierung
    df = df[['timestamp', 'ip', 'url', 'status_code']]
    print(f"✅ Rohdaten erfolgreich bereinigt. {len(df)} Einträge bereit zur Archivierung.")

except FileNotFoundError:
    print(f"FEHLER: Datei nicht gefunden unter {LOG_PATH}. Bitte stellen Sie sicher, dass `weblog.csv` im Verzeichnis `./data/` liegt.")
    raise


# --- DuckDB + DuckLake ---
con = duckdb.connect('forensic_duckdb.db')
con.sql("INSTALL ducklake;")
con.sql("LOAD ducklake;")

# Detach Katalog falls er bereits besteht
try:
    con.sql(f"DETACH {CATALOG_NAME};")
except:
    pass # Ignore Fehler falls Katalog ist nicht attached


# Katalog anhängen und verwenden (definiert den Speicherort der Snapshots)
attach_query = f"""
    ATTACH 'ducklake:{DUCKLAKE_METADATA_PATH}' AS {CATALOG_NAME}
    (DATA_PATH '{DUCKLAKE_DATA_PATH}');
"""
con.sql(attach_query)
con.sql(f"USE {CATALOG_NAME};")

print(f"✅ Forensisches Log-Archiv '{CATALOG_NAME}' initialisiert.")


# --- Snapshot 1 erstellen ---
con.sql(f"CREATE OR REPLACE TABLE {TABLE_NAME} AS SELECT * FROM df LIMIT 0;")
con.sql(f"INSERT INTO {TABLE_NAME} SELECT * FROM df;")

snapshot_info = con.sql(
    f"SELECT snapshot_id, snapshot_time "
    f"FROM ducklake_snapshots('{CATALOG_NAME}') "
    f"ORDER BY snapshot_id DESC LIMIT 1"
).fetchone()

# Store snapshot_1_id for later use
snapshot_1_id = snapshot_info[0]

print(f"\n✅ Snapshot 1 (Original-Log) erfasst. ID: {snapshot_1_id}, Time: {snapshot_info[1]}")

## 3. Simulierte Log-Manipulation und Snapshot 2 (Manipuliertes Log)


Es wird simuliert, dass ein Angreifer kritische Log-Einträge entfernt und einen Ablenkungseintrag hinzufügt. Diese Aktionen führen zum **Snapshot 2**, dem manipulierten Zustand. Kritische Log- Einträge sind hierbei alle 400er Fehlercodes, die auf eine ungültige Anfrage hinweisen. So könnte der Angreifer versuchen, seine Zugriffe zu vertuschen. Der Fehler **404 (Not Found)** könnte z.B. auch auf **Directory Scans** (Angriffe zur Pfaderkennung) hinweisen, was der Angreifer vermeiden möchte (https://infosecwriteups.com/investigate-web-attacks-challenge-lets-defend-24ea96524290). Im besten Falle für ihn bleibt sein Angriff/ Datenabgriff erst einmal unbekannt.

In [ ]:
# Kurze Pause für eindeutigen Snapshot-Zeitstempel
time.sleep(1)

# --- Simulierte Log-Manipulation (Beweisvertuschung) ---

# 1. Löschen verdächtiger Einträge (Alle Fehler-Einträge >= 400 werden gelöscht)
# Dies ist eine direkte SQL-Aktion, die das Log manipuliert.
deleted_count = con.execute(f"DELETE FROM {TABLE_NAME} WHERE status_code >= 400;").fetchall()[0][0]
print(f"   -> {deleted_count} Fehler-Einträge (>= 400) wurden gelöscht (Simulierte Manipulation).")

# 2. Hinzufügen eines unverdächtigen Eintrags im 'Rohdaten'-Format (Ablenkung)

# Um die Illusion der Rohdaten-Manipulation zu erzeugen, wird ein
# DataFrame erstellt, der die SPALTEN-NAMEN des Originals nutzt ('Time', 'Ip', 'Status', etc.),
# anstatt der bereits bereinigten Namen ('timestamp', 'status_code').

# Der vorherige Code hat die Spalten jedoch bereits in Kleinbuchstaben umbenannt.
# Daher nutzen werden hier BEREINIGTEN SPALTEN-NAMEN verwendet, aber die Werte
# werden so ausagefüllt, als ob sie aus dem Rohdaten-Kontext stammen (z.B. ein normaler 200er-Status).

new_log_df = pd.DataFrame({
    'timestamp': [pd.to_datetime(datetime.now())],
    'ip': ['203.0.113.5'],
    'url': ['/index.html'],
    'status_code': [200] # Numerische Statuscodes werden direkt unterstützt
})
con.sql(f"INSERT INTO {TABLE_NAME} SELECT * FROM new_log_df;")
print(f"   -> Ein neuer 200-Eintrag wurde hinzugefügt (Ablenkung).")

# --- Snapshot 2 erfassen ---
# Dieser Snapshot speichert den manipulierten Zustand in der History.
snapshot_2_id = con.sql(f"SELECT max(snapshot_id) FROM ducklake_snapshots('{CATALOG_NAME}')").fetchone()[0]
print(f"\n✅ Snapshot 2 (Manipuliertes Log) erfasst. ID: {snapshot_2_id}")

con.sql(f"SELECT snapshot_id, strftime(snapshot_time, '%Y-%m-%d %H:%M:%S') AS time FROM ducklake_snapshots('{CATALOG_NAME}') ORDER BY snapshot_id;").show()

## 4. Forensische Analyse: Beweissicherung mit DuckLake-Funktionen



In dem Code werden vier forensische Analyseschritte mit DuckLake durchgeführt:
Zuerst zeigt ducklake_snapshots die komplette Versionshistorie der Tabelle und macht sichtbar, wann Änderungen passiert sind.
Danach rekonstruiert ducklake_table_deletions, welche Einträge zwischen zwei Snapshots gelöscht wurden, um Manipulationen oder Vertuschungen nachzuweisen.
Anschließend zeigt ducklake_table_insertions, welche neuen Einträge hinzugefügt wurden, die als Ablenkung oder Verschleierung der Änderungen dienen könnten.
Zuletzt ruft ducklake_table_changes alle Änderungen zwischen zwei Snapshots ab und listet neben den Tabellenspalten auch Snapshot-ID, Row-ID sowie den Änderungstyp (insert oder delete) auf (https://duckdb.org/docs/stable/core_extensions/ducklake.html)

In [ ]:
# Die vier Hauptfunktionen der forensischen Analyse:

# 1. ducklake_snapshots: Zeigt die gesamte Versionshistorie
print("\n[Beweis 1: Manipulationshistorie]\n")
con.sql(f"""
    SELECT
        snapshot_id,
        strftime(snapshot_time, '%Y-%m-%d %H:%M:%S') AS time,
        changes
    FROM
        ducklake_snapshots('{CATALOG_NAME}')
    ORDER BY
        snapshot_id;
""").show()
print("Die Snapshots listen die vollständige **Transaktionshistorie** auf und dokumentieren exakt den **Originalzustand (1)** und den **manipulierten Zustand (2)** der Daten.")

# 2. ducklake_table_deletions: Zeigt die Vertuschung
print("\n[Beweis 2: Gelöschte Einträge (Vertuschung) zwischen Snapshot 1 und 2]\n")
con.sql(f"""
    SELECT
        ip,
        url,
        status_code
    FROM
        ducklake_table_deletions('{CATALOG_NAME}', 'main', '{TABLE_NAME}', {snapshot_1_id}, {snapshot_2_id});
""").show()
print("Die Deletions-Funktion rekonstruiert exakt die von der Manipulation entfernten Beweismittel (alle Statuscodes >= 400).")

# 3. ducklake_table_insertions: Zeigt die Ablenkung
print("\n[Beweis 3: Neu hinzugefügte Einträge (Ablenkung) zwischen Snapshot 1 und 2]\n")
con.sql(f"""
    SELECT
        ip,
        url,
        status_code
    FROM
        ducklake_table_insertions('{CATALOG_NAME}', 'main', '{TABLE_NAME}', {snapshot_1_id}, {snapshot_2_id});
""").show()
print("Die Ausgabe der zeigt alle Zeilen, die von den gelöschten Beweisen nicht betroffen waren, aber neu in den Datenbestand eingefügt werden mussten, um den neuen, manipulierten Zustand der Tabelle widerzuspiegeln.")

# 4. ducklake_table_changes: Zeigt alle Änderungen (Inserts & Deletes)
print("\n[Beweis 4: Alle Änderungen zwischen Snapshot 1 und 2]\n")
con.sql(f"""
    SELECT
        snapshot_id,
        rowid,
        change_type,
        ip,
        url,
        status_code
    FROM
        ducklake_table_changes(
            '{CATALOG_NAME}',
            'main',
            '{TABLE_NAME}',
            {snapshot_1_id},
            {snapshot_2_id}
        );
""").show()

print("Die Changes-Funktion listet alle eingefügten und gelöschten Zeilen auf und kennzeichnet sie mit 'insert' oder 'delete'.")

## 5. Forensische Statuscode-Analyse: Beweis der Datenmanipulation


Dieser Code vergleicht die **HTTP-Statuscode-Verteilung** des **aktuellen (manipulierten) Zustands** (Snapshot 2) mit den **gelöschten Einträgen** (die den Beweis der Manipulation darstellen). Es wird die `ducklake_table_deletions`-Funktion genutzt, um die forensischen Beweise direkt abzurufen. Die zwei Balkendiagrammen visualisieren einmal die manipulierte aktuelle Verteilung und einmal die gelöschten Fehlercodes als Beweis. Am Ende gibt der Code die forensisch relevanten gelöschten Statuscodes auch tabellarisch im Terminal aus.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Daten für den "Original (Beweis)" Zustand abrufen ---

# Die gelöschten Zeilen (deletions) entsprechen den Fehler-Einträgen,
# die in der Original-Logdatei enthalten waren.
df_deletions = con.execute(f"""
    SELECT
        CAST(status_code AS VARCHAR) AS status_code
    FROM
        ducklake_table_deletions('{CATALOG_NAME}', 'main', '{TABLE_NAME}', {snapshot_1_id}, {snapshot_2_id});
""").fetchdf()

# Analyse der gelöschten Einträge (die kritisch sind)
original_errors = df_deletions.groupby('status_code').size().reset_index(name='cnt')
original_errors['state'] = 'Original (Gelöschte Fehler)'
original_errors['version_id'] = 1


# --- 2. Daten für den "Aktuellen (Manipulierten)" Zustand abrufen ---
df_manipulated = con.execute(f"""
    SELECT
        CAST(status_code AS VARCHAR) AS status_code,
        COUNT(*) AS cnt,
        2 AS version_id,
        'Manipuliert (Aktuell)' AS state
    FROM
        {TABLE_NAME}
    GROUP BY
        status_code
""").fetchdf()


# --- 3. Visualisierung der relevanten Daten (Manipulation vs. Gelöschte Fehler) ---

# Kombinieren der Fehler aus dem Originalzustand und dem aktuellen Zustand
combined_analysis = pd.concat([original_errors, df_manipulated], ignore_index=True)

# Funktion zur Farbbestimmung
def get_colors(df):
    colors = []
    for s in df['status_code']:
        try:
            if int(s) >= 400:
                colors.append('darkred')
            else:
                colors.append('green')
        except:
            colors.append('gray')
    return colors

df_original_errors = combined_analysis[combined_analysis['version_id'] == 1].copy()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Manipulierter Zustand (Aktuell)
df_manipulated.plot.bar(x="status_code", y="cnt", legend=False, ax=axes[0], color=get_colors(df_manipulated))
axes[0].set_title("HTTP-Statuscode Verteilung (Aktuell - Manipuliert)")
axes[0].set_ylabel("Anzahl der Anfragen")
axes[0].set_xlabel("Statuscode")
axes[0].tick_params(axis='x', rotation=0)

# Plot 2: Historischer Beweis (Nur die gelöschten Fehlercodes)
df_original_errors.plot.bar(x="status_code", y="cnt", legend=False, ax=axes[1], color='darkred')
axes[1].set_title("HTTP-Statuscode Verteilung (Forensischer Beweis: Gelöschte Fehler)")
axes[1].set_ylabel("Anzahl der Anfragen")
axes[1].set_xlabel("Statuscode")
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nStatuscode-Analyse der gelöschten Einträge (Forensischer Beweis):")
print(df_original_errors)

Welche IP-Adressen haben 404-Einträgen im Originalzustand verursacht?

In [ ]:
# --- 4. Analyse: Von welchen IP-Adressen kamen die 404-Fehler im Originalzustand? ---

df_404_ips = con.execute(f"""
    SELECT
        ip,
        COUNT(*) AS cnt
    FROM
        ducklake_table_deletions('{CATALOG_NAME}', 'main', '{TABLE_NAME}', {snapshot_1_id}, {snapshot_2_id})
    WHERE
        status_code = 404
    GROUP BY
        ip
    ORDER BY
        cnt DESC
""").fetchdf()

print("Top IP-Adressen mit 404-Fehlern (Originalzustand, forensischer Beweis):")
print(df_404_ips)

# Visualisierung als Balkendiagramm
df_404_ips.plot.bar(x="ip", y="cnt", legend=False, figsize=(10, 5), title="IP-Adressen mit 404-Fehlern (Forensischer Beweis)")
plt.xlabel("IP-Adresse")
plt.ylabel("Anzahl der 404-Fehler")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 6. Weitere beispielhafte Analysen bei einer forensischen Analyse


Unabhängig der Snapshots könnten diese Analysen Aufschluss über den Angriff bieten.

 ### Häufig aufgerufene URLs
 Der Code ermittelt die Top 10 am häufigsten aufgerufenen URLs und gibt sie tabellarisch aus. Anschließend erstellt er ein Balkendiagramm, das die Aufrufzahlen dieser URLs gegenüberstellt. Dadurch wird sichtbar, welche Seiten besonders oft aufgerufen wurden und wie stark sie sich in der Verteilung unterscheiden.

In [ ]:
# Abfrage der Top 10 URLs aus dem aktuellen Zustand (Snapshot 2 - Manipuliert)
top_urls = con.execute(f"""
    SELECT
        url,
        COUNT(*) AS cnt
    FROM
        {TABLE_NAME}
    GROUP BY
        url
    ORDER BY
        cnt DESC
    LIMIT 10
""").fetchdf()

print("Top 10 URLs (Aktueller Zustand - Manipuliert):")
print(top_urls)

# Visualisierung
top_urls.plot.bar(x="url", y="cnt", legend=False, title="Top 10 URLs (Aktueller Zustand - Manipuliert)")
plt.xlabel("URL-Pfad")
plt.ylabel("Anzahl der Anfragen")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Interpretation:**  
Viele **Login- oder Admin-Seiten** können auf **Brute-Force- oder Enumeration-Angriffe** hinweisen (https://www.fortinet.com/uk/resources/cyberglossary/brute-force-attack) ; (https://www.techtarget.com/searchsecurity/tip/What-enumeration-attacks-are-and-how-to-prevent-them)


### Zeitliche Analyse (Requests pro Tag)

Der Code zählt die Anzahl der Requests pro Tag aus der Logtabelle, filtert dabei alle Einträge heraus, die nach 2019 (dort gibt es kaum Einträge mehr) liegen und gibt die Ergebnisse tabellarisch aus. Anschließend wird ein Liniendiagramm erstellt, das den zeitlichen Verlauf der Requests bis einschließlich 2019 darstellt.

In [ ]:
# Abfrage der Requests pro Tag, gefiltert auf das Jahr 2019 und älter (<= 2019)
requests_per_day = con.execute(f"""
    SELECT
        CAST(timestamp AS DATE) AS day,
        COUNT(*) AS cnt
    FROM
        {TABLE_NAME}
    WHERE
        -- Filtert alle Einträge aus, deren Jahr größer als 2019 ist
        EXTRACT(YEAR FROM timestamp) <= 2019
    GROUP BY
        day
    ORDER BY
        day
""").fetchdf()

print("Requests pro Tag (Aktueller Zustand - Gefiltert auf <= 2019):")
print(requests_per_day)

# Visualisierung des Liniendiagramms
requests_per_day.plot(
    x="day",
    y="cnt",
    kind="line",
    legend=False,
    title="Requests pro Tag (bis einschließlich 2019)"
)
plt.xlabel("Datum")
plt.ylabel("Anzahl der Requests")
plt.tight_layout()
plt.show()

### Top IP-Adressen

Der Code ermittelt die Top 10 IP-Adressen mit den meisten Anfragen und gibt sie tabellarisch aus. Danach stellt er diese Ergebnisse in einem Balkendiagramm dar, um die Verteilung der Anfragen pro IP visuell sichtbar zu machen.

In [ ]:
top_ips_manipulated = con.execute(f"""
    SELECT
        ip,
        COUNT(*) AS cnt
    FROM
        {TABLE_NAME}
    GROUP BY
        ip
    ORDER BY
        cnt DESC
    LIMIT 10
""").fetchdf()

print("Top 10 IPs (Aktueller, manipulierter Zustand):")
print(top_ips_manipulated)

# Visualisierung
top_ips_manipulated.plot.bar(x="ip", y="cnt", legend=False, title="Top 10 IPs (Aktueller Zustand - Manipuliert)")
plt.xlabel("IP-Adresse")
plt.ylabel("Anzahl der Anfragen")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---

## 7. Dokumentation und Abschluss


Nach Abschluss der forensischen Analyse werden die temporären Datenbank- und Dateisystemobjekte entfernt, um eine saubere Arbeitsumgebung zu gewährleisten.

In [ ]:
import shutil # Import the shutil module
import os # Import the os module

# Schließe die DuckDB-Verbindung
con.close()

# Entferne die temporär erstellten forensischen Artefakte und Daten (DuckDB- und DuckLake-Dateien)
if os.path.exists('forensic_duckdb.db'):
    os.remove('forensic_duckdb.db')

if os.path.exists(DUCKLAKE_METADATA_PATH):
    os.remove(DUCKLAKE_METADATA_PATH)

if os.path.exists(DUCKLAKE_DATA_PATH):
    shutil.rmtree(DUCKLAKE_DATA_PATH)

print("✅ Forensische Umgebung (Datenbank-Dateien und DuckLake-Katalog) erfolgreich bereinigt.")

# Hinweis zur Verwendung von generativer KI


Dieser Code wurde unter Verwendung eines KI-Modells (Google Gemini 2.5 Flash) zur Unterstützung entwickelt. Das KI-Modell wurde verwendet, um die Markdowns zu formatieren, Erklärungen bereitzustellen und Vorschläge für die Implementierung zu machen. Die endgültige Validierung, Anpassung und Verantwortung für die Korrektheit des Codes und der Erklärungen liegt bei der Autorin.
